In [7]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [8]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [9]:
df_green.registerTempTable('green')

In [10]:
df_green_revenue = spark.sql("""
SELECT 
    -- Revcenue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    hour, zone
ORDER BY 
    hour, zone
""")

In [11]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|            195.03|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25|             531.0|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|            317.27|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [12]:
df_green_revenue \
.repartition(20) \
.write.parquet('data/report/revenue/green', mode="overwrite")

In [13]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [14]:
df_yellow.registerTempTable('yellow')

In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Revcenue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records

FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    hour, zone
ORDER BY
    hour, zone
""")

In [16]:
df_yellow_revenue \
.repartition(20b) \
.write.parquet('data/report/revenue/yellow', mode="overwrite")

In [20]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [21]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how='outer')

In [24]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

Note: We are going to read the "materialized" table, so that subsquent use of the dataframe will not cause a recomputation of everithing from the first read of parquet green and yellow

In [38]:
df_join = spark.read.parquet('data/report/revenue/total')

In [39]:
df_zones = spark.read.parquet('zones/')

In [40]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [41]:
df_result

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint, LocationID: string, Borough: string, Zone: string, service_zone: string]

Note: zone and Zone. Spark will raise error when writing into parquet file (two columns with the same name 'zone')

Note: Default join type in Spark is inner

In [42]:
df_result.drop('LocationID', 'Zone').write.parquet('tmp/revenue-zones', mode='overwrite')

The above job will have only one stage because we are using materialized tables and the second table is small and hence, each excutor, when processing a partition, gets a copy of the table, so there is no resuffle to perform, just the task of join records.